In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [84]:
df = pd.read_csv('results.csv')

nn_df = df[df['name'] == 'nn'].reset_index()
gc_df = df[df['name'] == 'gc'].reset_index()
ls_random_df = df[df['name'] == 'ls-random'].reset_index()
ls_nn_df = df[df['name'] == 'ls-nn'].reset_index()
ils_df = df[df['name'] == 'ils'].reset_index()

nn_df.head(n=5)

,index,name,instance_name,primary_graph,secondary_graph,time_ns,primary_cost,secondary_cost,total_cost
0,100,nn,kroA100,15-87-21-93-69-52-78-17-23-37-35-98-83-9-71-20...,4-36-77-51-95-38-29-47-99-70-40-13-2-45-28-33-...,5452309,13228,18698,31926
1,101,nn,kroA100,28-45-2-42-13-70-40-99-47-51-77-95-4-36-32-75-...,64-65-3-96-55-79-30-88-41-7-91-0-62-5-48-89-9-...,4997223,12480,16404,28884
2,102,nn,kroA100,4-36-77-51-95-38-29-47-99-70-40-13-2-45-28-33-...,52-78-17-23-37-35-98-83-9-71-20-73-58-16-14-10...,3864575,14436,12789,27225
3,103,nn,kroA100,84-67-72-49-43-1-53-39-63-68-80-24-60-50-86-8-...,69-65-64-3-96-55-79-30-88-41-7-91-0-62-5-48-89...,3490109,11855,18696,30551
4,104,nn,kroA100,12-32-75-94-81-49-43-72-67-84-38-29-95-77-51-4...,59-61-76-22-97-90-44-31-10-14-16-73-20-58-71-9...,1857895,12651,16412,29063


In [91]:
def get_data(solver_df, instance):
    instance_df = solver_df[solver_df['instance_name'] == instance]
    instance_df = instance_df.reset_index()
    
    tc = instance_df['total_cost']
    best_cost = instance_df.iloc[tc.idxmin('column')]['total_cost']
    worst_cost = instance_df.iloc[tc.idxmax('column')]['total_cost']
    avg_cost = tc.mean()
    
    time = instance_df['time_ns']
    best_time = instance_df.iloc[time.idxmin('column')]['time_ns'] / 1_000_000
    worst_time = instance_df.iloc[time.idxmax('column')]['time_ns'] / 1_000_000
    avg_time = time.mean() / 1_000_000
    
    best_primary_graph = instance_df.iloc[tc.idxmin('column')]['primary_graph']
    best_secondary_graph = instance_df.iloc[tc.idxmin('column')]['secondary_graph']

    return {
        'best_result': {
            'primary': best_primary_graph,
            'secondary': best_secondary_graph
        },
        'best_cost': best_cost,
        'worst_cost': worst_cost,
        'avg_cost': avg_cost,
        'best_time': best_time,
        'worst_time': worst_time,
        'avg_time': avg_time
    }

In [93]:
get_data(ls_random_df, 'kroA150')

{'best_result': {'primary': '3-142-55-52-133-21-93-78-89-112-31-14-16-140-73-98-110-103-71-20-130-92-107-68-39-43-125-94-75-51-100-120-111-33-82-54-11-85-149-59-44-90-7-41-121-79-137-60-86-8-116-13-70-47-131-139-56-19-109-22-48-5-141-104-84-134-122-42-127-114-34-91-65-128-25',
  'secondary': '87-136-105-83-9-62-46-28-38-36-102-145-32-12-81-49-72-67-106-29-95-77-4-146-115-1-53-88-30-138-118-96-74-129-0-37-23-17-64-123-117-132-147-66-63-143-113-80-24-50-108-10-58-126-101-35-97-76-61-119-135-40-99-2-45-148-26-6-144-124-57-27-18-69-15'},
 'best_cost': 62194,
 'worst_cost': 85295,
 'avg_cost': 71767.55,
 'best_time': 200.55258,
 'worst_time': 471.029548,
 'avg_time': 294.563146}

In [94]:
results = {
    'nn_100': get_data(nn_df, 'kroA100'),
    'nn_150': get_data(nn_df, 'kroA150'),
    'gc_100': get_data(gc_df, 'kroA100'),
    'gc_150': get_data(gc_df, 'kroA150'),
    'ls_random_100': get_data(ls_random_df, 'kroA100'),
    'ls_random_150': get_data(ls_random_df, 'kroA150'),
    'ls_nn_100': get_data(ls_nn_df, 'kroA100'),
    'ls_nn_150': get_data(ls_nn_df, 'kroA150'),
    'ils_100': get_data(ils_df, 'kroA100'),
    'ils_150': get_data(ils_df, 'kroA150')
#     'ea': get_data(nn_df)
}

In [95]:
COLUMNS_COST = ['min_cost', 'max_cost', 'avg_cost']
COLUMNS_TIME = ['min_time', 'max_time', 'avg_time']

def to_series(data_dict, key, columns):
    series = []
    indexes = []
    
    for solver, values in data_dict.items():
        best = values['best_{}'.format(key)]
        worst = values['worst_{}'.format(key)]
        avg = values['avg_{}'.format(key)]
        
        indexes.append(solver)
        series.append(pd.Series([best, worst, avg], index=columns))
    
    return pd.DataFrame(series, index=indexes)

In [96]:
to_series(results, 'cost', COLUMNS_COST)

,min_cost,max_cost,avg_cost
nn_100,24458.0,32775.0,29202.78
nn_150,30664.0,38806.0,35038.48
gc_100,28450.0,36401.0,31729.88
gc_150,35039.0,43548.0,38458.19
ls_random_100,43370.0,69950.0,54133.23
ls_random_150,62194.0,85295.0,71767.55
ls_nn_100,47754.0,47754.0,47754.00
ls_nn_150,73733.0,73733.0,73733.00
ils_100,42989.0,50411.0,47713.75
ils_150,59896.0,61045.0,61033.51


In [97]:
to_series(results, 'time', COLUMNS_TIME)

,min_time,max_time,avg_time
nn_100,1.324470,16.957106,3.463314
nn_150,3.229817,4.356971,3.425506
gc_100,3.653104,115.813855,9.061932
gc_150,10.662524,12.045759,11.073706
ls_random_100,53.155598,280.841086,90.544357
ls_random_150,200.552580,471.029548,294.563146
ls_nn_100,81.763279,141.734193,89.455478
ls_nn_150,271.416264,314.174419,284.051172
ils_100,7090.464706,11180.122152,7766.326834
ils_150,27494.544112,44598.965537,32325.368920
